## 03-July-2024 == BM25 with Tim slides

#### Tim Spann @PaaSDev

### Milvus - Attu

![milvuslogo](https://milvus.io/images/milvus_logo.svg)


### CODE + COMMUNITY

Please join my meetup group NJ/NYC/Philly/Virtual. 

[https://www.meetup.com/unstructured-data-meetup-new-york/](https://www.meetup.com/unstructured-data-meetup-new-york/)


#### Contact Us

Get Milvused!  [https://milvus.io/](https://milvus.io/)

Read my Newsletter every week! [https://github.com/tspannhw/FLiPStackWeekly/blob/main/142-17June2024.md](https://github.com/tspannhw/FLiPStackWeekly/blob/main/142-17June2024.md)

For more cool Unstructured Data, AI and Vector Database videos check out the Milvus vector database videos here
[https://www.youtube.com/@MilvusVectorDatabase/videos](https://www.youtube.com/@MilvusVectorDatabase/videos)

#### Unstructured Data Meetups 

[https://www.meetup.com/pro/unstructureddata/](https://www.meetup.com/pro/unstructureddata/)
[https://zilliz.com/community/unstructured-data-meetup](https://zilliz.com/community/unstructured-data-meetup)
[https://zilliz.com/event](https://zilliz.com/event)

#### [https://x.com/milvusio](Twitter/X)  

#### [https://www.linkedin.com/company/zilliz/](LinkedIn)

#### [https://discord.com/invite/FjCMmaJng6](Discord)

#### [https://milvusio.medium.com/](Blog)

#### Please star our [https://github.com/milvus-io/milvus](Github)

#### [https://www.youtube.com/@FLaNK-Stack](Youtube)

#### [https://medium.com/@tspann/subscribe](Blog)

In [77]:
import os
from pymilvus import MilvusClient
from pymilvus import (
   utility,
   FieldSchema, CollectionSchema, DataType,
   Collection, AnnSearchRequest, RRFRanker, connections,
)
from pymilvus.model.sparse.bm25.tokenizers import build_default_analyzer
from pymilvus.model.sparse import BM25EmbeddingFunction
from pymilvus import model

DIMENSION = 64 
MILVUS_URL = "http://192.168.1.163:19530" 
COLLECTION_NAME = "traveladvisories"
TRAVEL_URL = "https://travel.state.gov/_res/rss/TAsTWs.xml"

# -----------------------------------------------------------------------------
# Connect to Milvus

# Local Docker Server
milvus_client = MilvusClient( uri=MILVUS_URL )

In [78]:
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import pprint

## schema
schema = milvus_client.create_schema(auto_id=True, enable_dynamic_fields=True)
schema.add_field(field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100)
schema.add_field(field_name="title", datatype=DataType.VARCHAR,  max_length=512)
schema.add_field(field_name="link", datatype=DataType.VARCHAR,  max_length=512)
schema.add_field(field_name="summary", datatype=DataType.VARCHAR,  max_length=50000)
schema.add_field(field_name="publisheddate", datatype=DataType.VARCHAR,  max_length=100)
schema.add_field(field_name="sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR)

## index
##index_params = milvus_client.prepare_index_params()

##index_params.add_index(
##   field_name="summaryvector",
##    index_type="SPARSE_INVERTED_INDEX",
##    metric_type="IP"
##)
#     index_params=index_params
## create collection
milvus_client.create_collection(
    collection_name = COLLECTION_NAME,
    schema=schema
)

res = milvus_client.get_load_state(
    collection_name = COLLECTION_NAME
)

print(res)

{'state': <LoadState: NotLoad>}


In [79]:
# Build Corpous

analyzer = build_default_analyzer(language="en")   

# Create corpus based samples from documents
corpus = [
   "Reissued after periodic review without changes. Exercise normal precautions in Bhutan. Read the country information page for additional information on travel to Bhutan. If you decide to travel to Bhutan: Enroll in the Smart Traveler Enrollment Program ( STEP ) to receive Alerts and make it easier to locate you in an emergency. Follow the Department of State on Facebook and Twitter . Review the Country Security Report for Bhutan. Visit the CDC page for the latest Travel Health Information related to your travel. Prepare a contingency plan for emergency situations. Review the Traveler’s Checklist",
   "Reissued with obsolete COVID-19 page links removed",
   "Exercise increased caution in Tajikistan due to terrorism, unexploded landmines, and occasional violence near the border with Kyrgyzstan",
   "Prepare a contingency plan for emergency situations. Review the Traveler’s Checklist",
   "If you decide to travel to Zambia: Enroll in the Smart Traveler Enrollment Program ( STEP ) to receive Alerts and make it easier to locate you in an emergency. Follow the Department of State on Facebook and Twitter . Follow the U.S. Embassy in Zambia on Facebook and Twitter . Review the Country Security Report for Zambia. Prepare a contingency plan for emergency situations.  Review the Traveler’s Checklist . Visit the CDC page for the latest Travel Health Information related to your travel",
   "Exercise normal precautions in Barbados. Read the country information page for additional information on travel to Barbados. If you decide to travel to Barbados",
]

# Use the analyzer to instantiate the BM25EmbeddingFunction
bm25_ef = BM25EmbeddingFunction(analyzer)

# Fit the model on the corpus to get the statstics of the corpus
bm25_ef.fit(corpus)


In [87]:
import feedparser
from bs4 import BeautifulSoup

# Function to remove tags
def remove_tags(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

# Lets Explore What our Data Looks like
# Travel Advisories
feed = feedparser.parse(TRAVEL_URL)

# print(feed)

summaries = [] 
titles = []
links = []
publisheddates = []

for post in feed.entries:
    try:
       print(post.title)
       summary = remove_tags(str(post.summary))
       summaries.append(summary)
       titles.append(str(post.title))
       links.append(str(post.link))
       publisheddates.append(str(post.published))
    except Exception as e: 
        print(e)

try:
   summaryvector = bm25_ef.encode_documents(summaries)
   i = 0
    
   #entities = [
   #     {
   #         "title": titles[i],
   #         "link": links[i],
   #         "summary": summaries[i],
   #         "publisheddate": publisheddates[i],
   #         "sparse_vector": embedding.todok(),
   #     }
   #     for embedding in summaryvector
   # ]

   entities = []
   i = 0
   for embedding in summaryvector:
        entities.append({
            "title": titles[i],
            "link": links[i],
            "summary": summaries[i],
            "publisheddate": publisheddates[i],
            "sparse_vector": embedding.todok(),
        })
        i += 1

    # for embedding in summaryvector
    #  for i in range(summaryvector.shape[0])
    # Insert entities
   res = milvus_client.insert(collection_name=COLLECTION_NAME, data=entities)
   # print(res)
except Exception as e: 
    print(e)

index_params = milvus_client.prepare_index_params()
index_params.add_index(
    field_name="sparse_vector",
    index_name="sparse_inverted_index",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="IP",
    params={"drop_ratio_build": 0.2},
)

milvus_client.create_index(collection_name=COLLECTION_NAME, index_params=index_params)


Bhutan - Level 1: Exercise Normal Precautions
Senegal - Level 1: Exercise Normal Precautions
Tajikistan - Level 2: Exercise Increased Caution
Iceland - Level 1: Exercise Normal Precautions
Mainland China, Hong Kong & Macau - See Summaries - Level 3: Reconsider Travel
Zambia - Level 1: Exercise Normal Precautions
Armenia - Level 2: Exercise Increased Caution
Syria - Level 4: Do Not Travel
Bolivia - Level 2: Exercise Increased Caution
Taiwan - Level 1: Exercise Normal Precautions
Rwanda - Level 1: Exercise Normal Precautions
Uruguay - Level 2: Exercise Increased Caution
Barbados - Level 1: Exercise Normal Precautions
North Korea - Level 4: Do Not Travel
Chad - Level 3: Reconsider Travel
Hungary - Level 1: Exercise Normal Precautions
Malta - Level 1: Exercise Normal Precautions
Burma (Myanmar) - Level 4: Do Not Travel
Qatar - Level 1: Exercise Normal Precautions
Saudi Arabia - Level 3: Reconsider Travel
Estonia - Level 1: Exercise Normal Precautions
British Virgin Islands - Level 1: Exerc

RPC error: [insert_rows], <MilvusException: (code=1100, message=the length (5479) of 7th string exceeds max length (4096): invalid parameter[expected=valid length string][actual=string length exceeds max length])>, <Time:{'RPC start': '2024-07-04 11:28:36.813465', 'RPC error': '2024-07-04 11:28:36.870016'}>


<MilvusException: (code=1100, message=the length (5479) of 7th string exceeds max length (4096): invalid parameter[expected=valid length string][actual=string length exceeds max length])>


In [3]:
!pip3 install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.4 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=ee9f7c228edf44b1ad3eaec8ee7c7422c44b552d3a2c0bbe75c69b9f39274883
  Stored in directory: /Users/timothyspann/Library/Caches/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [73]:
!pip3 install bs4

In [ ]:


search_results = milvus_client.search(
  collection_name=COLLECTION_NAME, # Collection name
  data=query_vector, # Replace with your query vector
  search_params={
    "metric_type": "IP"
  }, # Search parameters
  limit=10, # Max. number of search results to return
  output_fields=["pk","title","link","summary","publisheddate"], # Fields to return in the search results
  consistency_level="Eventually"
)

# Print search results
for hits in search_results:
    for hit in hits:
        print(f"Hit: {hit}")